In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
# from tqdm.auto import tqdm  # for notebooks

df = pd.DataFrame(np.random.randint(0, int(1e8), (10000, 1000)))

# Create and register a new `tqdm` instance with `pandas`
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()

# Now you can use `progress_apply` instead of `apply`
df.groupby(0).progress_apply(lambda x: x**2)

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
letters = ['A', 'B', 'D', 'E', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'Sade', 'Shin', 'T', 'Tet', 'U', 'W', 'Y', 'Z']
ur3 = []
for l in letters:
    response = requests.get("http://altorient.gko.uni-leipzig.de/SAD/" + l + ".htm")
    doc = response.content
    new_doc = UnicodeDammit.detwingle(doc)
    soup = BeautifulSoup(new_doc, 'html.parser')
    for n in soup.find_all("p"):
        if "Ur III" in n.get_text(): 
            ur3.append(n.get_text())

In [ ]:
ur3

In [ ]:
ur3_2 = []
for i in ur3:
    i = i.replace('â€�', '"')
    i = i.replace('â€œ', '"')
    i = i.replace('tÃºg', '{tug₂}')
    i = i.replace('\n', ' ')
    i = i.replace('Ãº', 'ú')
    ur3_2.append(i)

In [ ]:
ur3_2

In [ ]:
new_doc.decode("utf-8")

In [ ]:
import json

In [ ]:
d = '../2_1_Data_Acquisition_ORACC/jsonzip/obmc/corpusjson'
with open(d + '/X000011.json', encoding = 'utf-8') as j:
    data_json = json.load(j)

In [ ]:
data = data_json['cdl']
type(data)

In [ ]:
for l in data:
    if 'linkbase' in l:
        print(l['linkbase'])

In [ ]:
import zipfile
file = "../2_1_Data_Acquisition_ORACC/jsonzip/epsd2-literary.zip"    
z = zipfile.ZipFile(file)
st = z.read("epsd2/literary/corpusjson/Q000616.json").decode("utf-8") 
text = json.loads(st)
cdl = text['cdl']
for n in cdl:
    if 'linkbase' in n:
        linkbase = n['linkbase']

In [ ]:
linkbase